In [1]:
import findspark

In [2]:
findspark.init('/home/saadatullah/spark-3.3.0-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('Crs_Line').getOrCreate()

22/08/01 21:45:18 WARN Utils: Your hostname, saadatullah-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/08/01 21:45:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/01 21:45:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
data = spark.read.csv('examples/src/main/resources/cruise_ship_info.csv' , inferSchema=True , header=True) 

In [8]:
data.show()

+-----------+-----------+---+-------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+-------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6| 30.277|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6| 30.277|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26| 47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|  110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22| 70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15| 70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 70.367|     20.56|  8.55| 10.22|            34.23| 9.2|
|Fascination|   Carnival| 19| 70.367|     20.52|  8.55

In [10]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [11]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [12]:
from pyspark.ml.feature import StringIndexer

In [22]:
Indexer = StringIndexer(inputCol='Cruise_line' , outputCol='Indexed_Cruise_Line')
Indexed = Indexer.fit(data).transform(data)
Indexed.show()

+-----------+-----------+---+-------+----------+------+------+-----------------+----+-------------------+
|  Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|Indexed_Cruise_Line|
+-----------+-----------+---+-------+----------+------+------+-----------------+----+-------------------+
|    Journey|    Azamara|  6| 30.277|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|      Quest|    Azamara|  6| 30.277|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|Celebration|   Carnival| 26| 47.262|     14.86|  7.22|  7.43|             31.8| 6.7|                1.0|
|   Conquest|   Carnival| 11|  110.0|     29.74|  9.53| 14.88|            36.99|19.1|                1.0|
|    Destiny|   Carnival| 17|101.353|     26.42|  8.92| 13.21|            38.36|10.0|                1.0|
|    Ecstasy|   Carnival| 22| 70.367|     20.52|  8.55|  10.2|            34.29| 9.2|                1.0|
|    Elation|   Carnival| 15| 70.367|     20.5

In [27]:
Indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Indexed_Cruise_Line']

In [28]:
Indexed.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Indexed_Cruise_Line: double (nullable = false)



In [29]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [58]:
assembler = VectorAssembler(inputCols=['passengers',
 'cabins',
 'passenger_density',
 'Indexed_Cruise_Line' , ], outputCol='features')

In [59]:
output = assembler.transform(Indexed)

In [60]:
output.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.277, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Indexed_Cruise_Line=16.0, features=DenseVector([6.94, 3.55, 42.64, 16.0]))

In [61]:
output.head(1)[0]

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.277, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Indexed_Cruise_Line=16.0, features=DenseVector([6.94, 3.55, 42.64, 16.0]))

In [62]:
for item in output.head(1)[0]:
    print(item)

Journey
Azamara
6
30.277
6.94
5.94
3.55
42.64
3.55
16.0
[6.94,3.55,42.64,16.0]


In [63]:
from pyspark.ml.regression import LinearRegression

In [64]:
final_data_to_work = output.select(['crew' , 'features'])

In [65]:
final_data_to_work.show()

+----+--------------------+
|crew|            features|
+----+--------------------+
|3.55|[6.94,3.55,42.64,...|
|3.55|[6.94,3.55,42.64,...|
| 6.7|[14.86,7.43,31.8,...|
|19.1|[29.74,14.88,36.9...|
|10.0|[26.42,13.21,38.3...|
| 9.2|[20.52,10.2,34.29...|
| 9.2|[20.52,10.2,34.29...|
| 9.2|[20.56,10.22,34.2...|
| 9.2|[20.52,10.2,34.29...|
|11.5|[37.0,14.87,29.79...|
|11.6|[29.74,14.87,36.9...|
| 6.6|[14.52,7.26,31.72...|
| 9.2|[20.52,10.2,34.29...|
| 9.2|[20.52,10.2,34.29...|
| 9.3|[21.24,10.62,40.4...|
|11.6|[29.74,14.87,36.9...|
|10.3|[21.24,10.62,41.6...|
| 9.2|[20.52,10.2,34.29...|
| 9.3|[21.24,11.62,41.6...|
| 9.2|[20.52,10.2,34.29...|
+----+--------------------+
only showing top 20 rows



In [66]:
train_data , test_data = final_data_to_work.randomSplit([0.7 , 0.3])

In [67]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               100|
|   mean| 7.751800000000002|
| stddev|3.6963532069193588|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [68]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                58|
|   mean| 7.867241379310346|
| stddev|3.1732289123521387|
|    min|              0.59|
|    max|             13.13|
+-------+------------------+



In [69]:
lr = LinearRegression(labelCol='crew' , featuresCol='features')

In [73]:
lr_model = lr.fit(train_data)

22/08/01 22:43:22 WARN Instrumentation: [aa4890af] regParam is zero, which might cause numerical instability and overfitting.


In [74]:
test_result = lr_model.evaluate(test_data)

In [75]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.8189077479029171|
| 0.09695832959461081|
| -0.5313675206111113|
| -0.4996379249290319|
|-0.45431027688982084|
|  -0.164384030444086|
| -1.0815231155667187|
| 0.05996180646639537|
| 0.06435814874186052|
| 0.06435814874186052|
|-0.04525409585768081|
|  1.9497751912132482|
| 0.16992778742599324|
|-0.11970784721022731|
|-0.29796088497920437|
| -0.5466028108127094|
| -1.4718268514731223|
|  0.3273963336085419|
| -1.4580463806130322|
|  0.4136156026159261|
+--------------------+
only showing top 20 rows



In [76]:
test_result.r2

0.9290026292531715

In [77]:
test_result.rootMeanSquaredError

0.8381967383831791

In [78]:
unlabelled_data = test_data.select('features')

In [79]:
unlabelled_data.show()

+--------------------+
|            features|
+--------------------+
|[0.66,0.33,50.62,...|
|[3.94,0.88,31.73,...|
|[2.96,1.48,56.76,...|
| [3.2,1.6,60.0,10.0]|
|[9.52,3.83,17.7,7.0]|
|[4.9,2.45,67.35,1...|
|[8.26,4.25,26.73,...|
|[7.76,3.86,32.22,...|
|[6.84,3.42,44.26,...|
|[6.84,3.42,44.26,...|
|[8.08,4.1,35.19,5.0]|
|[7.0,1.82,65.71,1...|
|[11.78,5.3,38.2,8.0]|
|[12.14,6.07,27.94...|
|[12.64,6.32,43.87...|
|[14.94,7.47,36.06...|
|[17.48,8.74,29.04...|
|[13.2,6.6,45.19,3.0]|
|[24.35,10.0,32.23...|
|[13.66,6.87,34.57...|
+--------------------+
only showing top 20 rows



In [80]:
predictions = lr_model.transform(unlabelled_data)

In [81]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[0.66,0.33,50.62,...| 1.408907747902917|
|[3.94,0.88,31.73,...|1.3630416704053891|
|[2.96,1.48,56.76,...|2.5013675206111112|
| [3.2,1.6,60.0,10.0]| 2.609637924929032|
|[9.52,3.83,17.7,7.0]| 3.424310276889821|
|[4.9,2.45,67.35,1...| 3.404384030444086|
|[8.26,4.25,26.73,...| 4.581523115566719|
|[7.76,3.86,32.22,...|3.7900381935336047|
|[6.84,3.42,44.26,...|3.9356418512581395|
|[6.84,3.42,44.26,...|3.9356418512581395|
|[8.08,4.1,35.19,5.0]| 4.045254095857681|
|[7.0,1.82,65.71,1...|2.5202248087867516|
|[11.78,5.3,38.2,8.0]| 5.030072212574007|
|[12.14,6.07,27.94...| 5.419707847210227|
|[12.64,6.32,43.87...| 5.867960884979205|
|[14.94,7.47,36.06...|  6.66660281081271|
|[17.48,8.74,29.04...| 7.611826851473122|
|[13.2,6.6,45.19,3.0]|6.1126036663914585|
|[24.35,10.0,32.23...| 8.058046380613032|
|[13.66,6.87,34.57...| 6.286384397384074|
+--------------------+------------

In [82]:
test_data.show()

+----+--------------------+
|crew|            features|
+----+--------------------+
|0.59|[0.66,0.33,50.62,...|
|1.46|[3.94,0.88,31.73,...|
|1.97|[2.96,1.48,56.76,...|
|2.11| [3.2,1.6,60.0,10.0]|
|2.97|[9.52,3.83,17.7,7.0]|
|3.24|[4.9,2.45,67.35,1...|
| 3.5|[8.26,4.25,26.73,...|
|3.85|[7.76,3.86,32.22,...|
| 4.0|[6.84,3.42,44.26,...|
| 4.0|[6.84,3.42,44.26,...|
| 4.0|[8.08,4.1,35.19,5.0]|
|4.47|[7.0,1.82,65.71,1...|
| 5.2|[11.78,5.3,38.2,8.0]|
| 5.3|[12.14,6.07,27.94...|
|5.57|[12.64,6.32,43.87...|
|6.12|[14.94,7.47,36.06...|
|6.14|[17.48,8.74,29.04...|
|6.44|[13.2,6.6,45.19,3.0]|
| 6.6|[24.35,10.0,32.23...|
| 6.7|[13.66,6.87,34.57...|
+----+--------------------+
only showing top 20 rows

